In [118]:
import os, json
from dotenv import load_dotenv
import random
load_dotenv()
import datetime
from tqdm import tqdm

In [ ]:
raw_dr_ntu_dir='./raw_dr_ntu'
raw_faculty_db = 'scse_profile'

process_faculty_db_dir ='./processed'
process_faculty_db = 'scse_profile'
process_co_author_db = 'google_scholar_co_author'

raw_google_scholar_dir = './raw_google_scholar'
raw_google_search_dir = './google_search'

process_publications_dir = './processed_google_scholar_publications'

research_interest_dir = './research_interest'

cur_year = datetime.datetime.now().year

In [ ]:
faculties = pd.read_csv(os.path.join(process_faculty_db_dir, process_faculty_db+'.csv'))
google_scholar_faculties = faculties[faculties['google_scholar'].notna()]

# Using OpenAI directly

In [129]:
import openai

def extract_topics(title, description):
    system_msg = "I am an AI assistant designed to help you extract the related field of computer science from a given text. \
    I understand that the text contains a title and a description of an academic paper and I am capable of identifying the field of computer science the paper is addressing. \
    I understand that I am only allowed to extract topics related to in the field of computer science and the topics must be generic. \
    Examples of generic fields of computer science are 'Distributed Systems', 'Computer Vision', 'Graph Mining', 'Erasure Coding', 'Bioinformatics', 'Operating Systems', 'Information Retrieval'. \
    The topics must be related to computer science. \
    I also understand that after extracting the topic, \
    My response must be a must be wrapped in `<result>`, eg: `<result>topic<result>` replace topic with the topic that sums up the academic paper."

    prompt = f"""
    Based on the title and description below, label the academic paper with the main field of computer science that it is addressing. 
    Your responsemust be a must be wrapped in `<result>`, eg: `<result>topic<result>` replace topic with the topic that sums up the academic paper.

    Title:

    {title}

    Description:

    {description}
    """
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "system", "content": system_msg },
                        {"role": "user", "content": prompt}
                ],
                request_timeout=10
            ) 
            output = response['choices'][0]['message']['content']
            if output.startswith('<result>') and output.endswith('<result>'):
                output = output.strip('<result>')
                return output
        except Exception as e:
            pass

In [ ]:
import openai

def group_topics(keywords):
    system_msg = "I am an AI assistant designed to help you group related keywords together. \
    I understand that I will receive a list of computer science realted keywords and I am able to understand each keyword. \
    My response be a must a list of lists eg: `[1,2,3,4],[q,w,e]` where within each list consist of keywords that are related to each other "

    prompt = f"""
    Based on the list of keywords provided below, help me to group them. 
    Your response must be a list of list where within each list are keywords that are related to each other.
    Wrap the list of list in `<result>`. eg `<result>[[1,2,3,4],[q,w,e]]<result>`.

    Keywords:

    {keywords}
    """
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[{"role": "system", "content": system_msg },
                        {"role": "user", "content": prompt}
                ],
                request_timeout=60*15,
                temperature=0.2
            ) 
            output = response['choices'][0]['message']['content']
            if output.startswith('<result>') and output.endswith('<result>'):
                output = output.strip('<result>')
                return output
        except Exception as e:
            pass

In [ ]:
import openai

def get_1_topic(keywords):
    system_msg = "I am an AI assistant designed to help you identify keyword for a list of words. \
    I understand that I will receive a list of computer science related keywords and I am able to understand each keyword. \
    My response be a must be wrapped in <result>, eg: <result>topic<result> replace topic with the topic that sums up the list of keywords. "

    prompt = f"""
    Based on the list of keywords provided below, help me to extract the main keyword. 
    Your response be a must be wrapped in <result>, eg: <result>topic<result> replace topic with the topic that sums up the list of keywords.

    Keywords:

    {','.join(keywords)}
    """
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[{"role": "system", "content": system_msg },
                        {"role": "user", "content": prompt}
                ],
                # request_timeout=10,
                temperature=1
            ) 
            output = response['choices'][0]['message']['content']
            if output.startswith('<result>') and output.endswith('<result>'):
                output = output.strip('<result>')
                return output
        except Exception as e:
            pass

# Script

In [ ]:


for i, row in tqdm(google_scholar_faculties.iterrows()):
    name, url, id = row['full_name'], row['google_scholar'], row['google_scholar_id']

    pub_file = f"{process_publications_dir}/{id}.json"
    
    if os.path.exists(pub_file):
        with open(pub_file, 'r') as f:
            pubs = json.load(f)

        topics = []
        for pub in tqdm(pubs):
            if pub['description'] is not None and pub['publication_year'] is not None and int(pub['publication_year'])>=min_year:
                topic = extract_topics(pub['title'], pub['description'])

                results = {
                    'link': pub['link'],
                    'topic': topic,
                }
                topics.append(results)

        
            with open(f"{research_interest_dir}/{id}.json  ", 'w') as f:
                json.dump(topics, f)


    

# Grouping Topics

In [ ]:

all_topics = {}
for i, row in tqdm(google_scholar_faculties.iterrows()):
    name, url, id = row['full_name'], row['google_scholar'], row['google_scholar_id']

    topics_file = f"{research_interest_dir}/{id}.json"
    
    if os.path.exists(topics_file):
        with open(topics_file, 'r') as f:
            topics = json.load(f)
        
        for topic in topics:
            all_topics[topic['topic']] = all_topics.get(topic['topic'], 0) + 1

len(topic)

In [190]:
import openai

def group_topics(keywords):
    system_msg = "I am an AI assistant designed to help you group related keywords together. \
    I understand that I will receive a list of computer science realted keywords and I am able to understand each keyword. \
    My response be a must a list of lists eg: `[1,2,3,4],[q,w,e]` where within each list consist of keywords that are related to each other "

    prompt = f"""
    Based on the list of keywords provided below, help me to group them. 
    Your response must be a list of list where within each list are keywords that are related to each other.
    Wrap the list of list in `<result>`. eg `<result>[[1,2,3,4],[q,w,e]]<result>`.

    Keywords:

    {keywords}
    """
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[{"role": "system", "content": system_msg },
                        {"role": "user", "content": prompt}
                ],
                request_timeout=60*15,
                temperature=0.2
            ) 
            output = response['choices'][0]['message']['content']
            if output.startswith('<result>') and output.endswith('<result>'):
                output = output.strip('<result>')
                return output
        except Exception as e:
            pass

In [ ]:
all_outputs = []
topics = list(all_topics.keys())
for i in tqdm(range(0, len(topics)-50, 50)):
    output = group_topics(topics[i:i+50])
    all_outputs.append(output)

all_outputs.append(group_topics(topics[len(topics)-50-1:]))

In [163]:
all_outputs = [x.strip('[').strip(']') for x in all_outputs]
all_outputs = [x.split('],')for x in all_outputs]

groupings = []
for x in all_outputs:
    for y in x:
        z = y.strip()
        z = z.strip('[').strip(']').split(',')
        group = []
        for a in z:
            b = a.strip().strip("'")
            group.append(b)
        groupings.append(group)
            

In [174]:
import openai

def get_1_topic(keywords):
    system_msg = "I am an AI assistant designed to help you identify keyword for a list of words. \
    I understand that I will receive a list of computer science related keywords and I am able to understand each keyword. \
    My response be a must be wrapped in <result>, eg: <result>topic<result> replace topic with the topic that sums up the list of keywords. "

    prompt = f"""
    Based on the list of keywords provided below, help me to extract the main keyword. 
    Your response be a must be wrapped in <result>, eg: <result>topic<result> replace topic with the topic that sums up the list of keywords.

    Keywords:

    {','.join(keywords)}
    """
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[{"role": "system", "content": system_msg },
                        {"role": "user", "content": prompt}
                ],
                # request_timeout=10,
                temperature=1
            ) 
            output = response['choices'][0]['message']['content']
            if output.startswith('<result>') and output.endswith('<result>'):
                output = output.strip('<result>')
                return output
        except Exception as e:
            pass

In [175]:
keyword = []

for group in groupings:
    if len(group)==1:
        output = group[0]
    else:
        output = get_1_topic(group)
    keyword.append(tuple([output, group]))


<result>Advanced Computer Sciences</result>
<result>Geospatial Technology</result>
<result>Federated Learning</result>
<result>Spatial Data Processing</result>
<result>Natural Language Processing</result>
<result>Multimodal Fusion</result>


In [ ]:
for i, row in tqdm(google_scholar_faculties.iterrows()):
    name, url, id = row['full_name'], row['google_scholar'], row['google_scholar_id']

    topics_file = f"{research_interest_dir}/{id}.json"
    
    if os.path.exists(topics_file):
        with open(topics_file, 'r') as f:
            pubs = json.load(f)
        
        for pub in pubs:
            topic = pub['topic']
            for key in keyword:
                for interest in key[1]:
                    if topic in interest:
                        pub['topic'] = key[0]
        
        with open(topics_file, 'w') as f:
            json.dump(pubs, f)

In [188]:
all_topics = {}
for i, row in tqdm(google_scholar_faculties.iterrows()):
    name, url, id = row['full_name'], row['google_scholar'], row['google_scholar_id']

    topics_file = f"{research_interest_dir}/{id}.json"
    
    if os.path.exists(topics_file):
        with open(topics_file, 'r') as f:
            topics = json.load(f)
        
        for topic in topics:
            all_topics[topic['topic']] = all_topics.get(topic['topic'], 0) + 1

len(topic)

118

In [192]:
agg_topic = group_topics(list(all_topics.keys()))

In [231]:
output = agg_topic
output = output.strip('[').strip(']')
output = output.split('],')
groupings = []
for x in output:
    z = x.strip()
    z = z.strip('[').strip(']').split(',')
    group = []
    for a in z:
        b = a.strip().strip("'")
        group.append(b)
    groupings.append(group)

In [232]:
keyword = []

for group in groupings:
    if len(group)==1:
        output = group[0]
    else:
        output = get_1_topic(group)
    keyword.append(tuple([output, group]))

<result>Computing</result>
<result>Federated Learning</result>


In [ ]:
for i, row in tqdm(google_scholar_faculties.iterrows()):
    name, url, id = row['full_name'], row['google_scholar'], row['google_scholar_id']

    topics_file = f"{research_interest_dir}/{id}.json"
    
    if os.path.exists(topics_file):
        with open(topics_file, 'r') as f:
            pubs = json.load(f)
        
        for pub in pubs:
            topic = pub['topic']
            for key in keyword:
                for interest in key[1]:
                    if topic in interest:
                        pub['topic'] = key[0]
        
        with open(topics_file, 'w') as f:
            json.dump(pubs, f)